In [3]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.2 MB/s eta 0:00:00


In [4]:
import streamlit as st

# ---------- ISA (aligned with your decoder) ----------
ISA = {
    "LDA": (0x1, True),   # 0001
    "LDB": (0x2, True),   # 0010
    "ADD": (0x3, False),  # 0011
    "SUB": (0x4, False),  # 0100
    "ST" : (0x5, True),   # 0101
    "CMP": (0x6, False),  # 0110
    "JMP": (0x7, True),   # 0111
    "HLT": (0xE, False),  # 1110
}

def bin8(n): return format(n & 0xFF, "08b")
def hex2(n): return format(n & 0xFF, "02X")

def rom_to_logisim_hex(rom):
    """Convert ROM into Logisim grid format (2 rows × 8 values)."""
    out = []
    for i in range(0, len(rom), 8):
        chunk = rom[i:i+8]
        out.append(" ".join(f"{v:02X}" for v in chunk))  # force 2-digit hex
    return "\n".join(out)

def assemble(src: str):
    rom = [0] * 16   # fixed 16x8 ROM
    pc = 0
    listing = []

    for line in src.splitlines():
        line = line.strip()
        if not line:
            continue

        parts = line.split()
        mnem = parts[0].upper()

        # ---------- Handle DATA ----------
        if mnem == "DATA":
            if len(parts) < 3:
                continue
            addr, val = int(parts[1]), int(parts[2])
            if not (0 <= addr < len(rom)):
                raise ValueError(f"DATA address {addr} out of range (0-{len(rom)-1})")
            rom[addr] = val & 0xFF
            listing.append([f"{addr:04b}", bin8(val), f"DATA {addr} {val}"])
            continue

        # ---------- Handle ISA ----------
        if mnem in ISA:
            opcode, has_operand = ISA[mnem]
            imm = 0
            if has_operand and len(parts) > 1 and parts[1].isdigit():
                imm = int(parts[1])

            # force no-operand instructions to 0000
            if mnem in ["ADD", "SUB", "CMP", "HLT"]:
                imm = 0

            code = (opcode << 4) | (imm & 0xF)
            if pc >= len(rom):
                raise ValueError("Program too large for 16x8 ROM")
            rom[pc] = code
            listing.append([f"{pc:04b}", bin8(code), f"{mnem} {imm if has_operand else ''}"])
            pc = (pc + 1) & 0xF
            continue

        # ---------- Ignore junk ----------
        continue

    return listing, rom

# ---------- Streamlit UI ----------
st.title("SAP-1 Smart Compiler App")

default_src = """LDA 7
LDB 8
ADD
ST 9
HLT

DATA 7 10
DATA 8 5
DATA 9 0
"""

src = st.text_area("Assembly Program", value=default_src, height=300)

if st.button("Compile"):
    try:
        listing, rom = assemble(src)

        st.subheader("Assembler Listing")
        st.table(listing)

        st.subheader("ROM Contents (16x8, linear)")
        st.text(" ".join(f"{v:02X}" for v in rom))   # fixed 2-digit hex

        st.subheader("Logisim ROM Paste Format")
        logisim_hex = rom_to_logisim_hex(rom)
        st.text(logisim_hex)

        # Download buttons
        st.download_button("Download binary", data="\n".join(bin8(v) for v in rom), file_name="binary.txt")
        st.download_button("Download hex", data="\n".join(f"{v:02X}" for v in rom), file_name="hex.txt")
        st.download_button("Download Logisim ROM", data=logisim_hex, file_name="logisim_rom.txt")

    except Exception as e:
        st.error("Compilation failed: " + str(e))


2025-09-28 06:50:06.311 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 06:50:06.464 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-28 06:50:06.465 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 06:50:06.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 06:50:06.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 06:50:06.467 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 06:50:06.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 06:50:06.469 Thread 'MainThread': mi